# 한국은행 경제통계 시스템 
(https://ecos.bok.or.kr/#/EconomicIndicatorExplanation)
## 주요통계해설
### FAQ

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
import time
from bs4 import BeautifulSoup
import pandas as pd
import re
from urllib.request import urlopen
import os
from PIL import Image

In [5]:
# 총 질문 갯수

driver = webdriver.Chrome()

driver.get("https://ecos.bok.or.kr/#/EconomicIndicatorExplanation")
time.sleep(5)
num = 0
for i in range(1, 13):

    # 대타이틀 클릭
    big_title = driver.find_element(By.XPATH, f'//*[@id="root"]/div[5]/div/div/div[1]/ul/li[{i}]/button')
    big_title.click()
    time.sleep(3)

    # big_title_text -> 제목 
    big_title_text = big_title.text
    # print("제목", big_title_text)

    # FAQ버튼 클릭
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(1) 
    tap_faq = driver.find_element(By.XPATH, f'//*[@id="root"]/div[5]/div/div/div[2]/div/div[1]/ul/li[5]/button')
    tap_faq.click()
    time.sleep(2)

    faq_text = driver.find_element(By.XPATH, '//*[@id="root"]/div[5]/div/div/div[2]/div/div[2]').text
    faq_num= driver.find_element(By.XPATH, '//*[@id="root"]/div[5]/div/div/div[2]/div/div[2]/div/div[1]/span').text
    num += int(faq_num)
    print(big_title_text, faq_num, num)

print("총 질문 갯수 : ", num)


국민소득통계 16 16
산업연관표 5 21
국민대차대조표 5 26
경기판단지표 17 43
기업경영분석지표 10 53
자금순환표 9 62
통화지표 7 69
금리지표 2 71
물가지수 6 77
국제수지표 4 81
국제투자대조표 8 89
무역지수 및 교역조건지수 5 94
총 질문 갯수 :  94


In [10]:
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(options=chrome_options)


driver.get("https://ecos.bok.or.kr/#/EconomicIndicatorExplanation")
time.sleep(3)

img_count = 1
add_info_list = []
len_html = 0
text_list = []
key_order = ["제목", "질문", "답변", "metas"]

def is_fully_visible(driver, element):
    '''
    element(table)가 laptop 화면에서 완전히 보이는지 잘려서 보이는지를 반환해주는 함수
    완전히 보일 시 -> True 반환
    잘려서 보일 시 -> False 반환
    '''
    return driver.execute_script(
        "var rect = arguments[0].getBoundingClientRect();"
        "return (rect.top >= 0 && rect.left >= 0 && "
        "rect.bottom <= (window.innerHeight || document.documentElement.clientHeight) && "
        "rect.right <= (window.innerWidth || document.documentElement.clientWidth));",
        element
    )

def capture_table_with_zoom(driver, element, min_zoom=50, default_zoom=100):
    '''
    테이블이 완전히 보일 때까지 zoom을 조정하며 스크린샷을 찍는 함수
    zoom 값이 50%까지 줄어들며, 그 후 100%로 되돌아갑니다.
    '''
    # 테이블을 중앙으로 스크롤
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'})", element)

    zoom = default_zoom  # 시작 zoom 값은 100%
    while zoom >= min_zoom:
        # Zoom을 설정
        driver.execute_script(f"document.body.style.zoom='{zoom}%'")
        
        # element가 완전히 보일 때까지 확인
        if is_fully_visible(driver, element):
            # 테이블이 완전히 보이면 캡쳐
            print("테이블이 {zoom}%로 완전히 보임. 캡처 중...")
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'})", element)
            element.screenshot(f"img_{img_count:03}_{k:03}.png")
            break  # 테이블이 보이면 루프 종료
        
        # Zoom을 줄여서 다시 확인
        print("테이블이 보이지 않아, zoom 중...")
        zoom -= 5  # zoom을 5%씩 줄여가며 테스트

    # 기본 zoom으로 돌아가기
    driver.execute_script(f"document.body.style.zoom='{default_zoom}%'")

for i in range(12):
    # 왼쪽 nav bar 클릭하기
    left_nav = driver.find_element(By.XPATH, f'//*[@id="root"]/div[5]/div/div/div[1]/ul/li[{i + 1}]/button')
    driver.execute_script("arguments[0].click();", left_nav)
    title = left_nav.text
    time.sleep(3)
    # FAQ 탭 클릭
    explanation = driver.find_element(By.XPATH, '//*[@id="root"]/div[5]/div/div/div[2]/div/div[1]/ul/li[5]/button')
    driver.execute_script("arguments[0].click();", explanation)
    time.sleep(1)
    # <ul class='ex-faq-list'> 내부에서 모든 <li class='ex-faq-item'> 요소 찾기
    ex_faq_items = driver.find_elements(By.XPATH, "//ul[@class='ex-faq-list']//li[@class='ex-faq-item']")

    # 각 토글 active 상태로 변경
    for ex_faq_item in ex_faq_items:
        # "active" 클래스가 없으면 클릭하여 활성화
        if 'active' not in ex_faq_item.get_attribute('class'):
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'})", ex_faq_item)
            ex_faq_item.click()
            time.sleep(1)
    
    
    # 테이블 스샷찍어 저장
    for body in driver.find_elements(By.CLASS_NAME, "answer-inner"):
        for k, tb_img in enumerate(body.find_elements(By.XPATH, ".//table | .//img"), start=1):
            if tb_img.tag_name == "table":
                print(title, "테이블 >>", f"img_{img_count:03}_{k:03}.png")
                # 테이블이 완전히 보일 때까지 zoom을 조정하며 캡쳐
                capture_table_with_zoom(driver, tb_img, min_zoom=50, default_zoom=100)
            else:
                pass
        img_count += 1



국민소득통계 테이블 >> img_005_001.png
테이블이 {zoom}%로 완전히 보임. 캡처 중...
국민소득통계 테이블 >> img_009_002.png
테이블이 {zoom}%로 완전히 보임. 캡처 중...
산업연관표 테이블 >> img_017_001.png
테이블이 {zoom}%로 완전히 보임. 캡처 중...


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=133.0.6943.142)
Stacktrace:
	GetHandleVerifier [0x00007FF6DDFBC6A5+28789]
	(No symbol) [0x00007FF6DDF25B20]
	(No symbol) [0x00007FF6DDDB8F9A]
	(No symbol) [0x00007FF6DDDA4D75]
	(No symbol) [0x00007FF6DDDC9784]
	(No symbol) [0x00007FF6DDE3F0BF]
	(No symbol) [0x00007FF6DDE5F0D2]
	(No symbol) [0x00007FF6DDE37103]
	(No symbol) [0x00007FF6DDDFFFC0]
	(No symbol) [0x00007FF6DDE01273]
	GetHandleVerifier [0x00007FF6DE301AED+3458237]
	GetHandleVerifier [0x00007FF6DE31829C+3550316]
	GetHandleVerifier [0x00007FF6DE30DB9D+3507565]
	GetHandleVerifier [0x00007FF6DE082C6A+841274]
	(No symbol) [0x00007FF6DDF309EF]
	(No symbol) [0x00007FF6DDF2CB34]
	(No symbol) [0x00007FF6DDF2CCD6]
	(No symbol) [0x00007FF6DDF1C119]
	BaseThreadInitThunk [0x00007FFDA0297034+20]
	RtlUserThreadStart [0x00007FFDA0642651+33]


In [ ]:
eco_stat_faq = pd.DataFrame(index = range(100), columns=["data_id", "data_file", "data_title", "data_source", "category_main", "category_sub", "data_type", "collected_date", "source_url" ,"content", "add_info"])
eco_stat_faq["data_id"] = [str(i + 1) for i in range(100)]
eco_stat_faq["data_file"] = "eco_stat_faq.jsonl"
eco_stat_faq["data_source"] = "한국은행 경제통계시스템"
eco_stat_faq["category_main"] = "경제"
eco_stat_faq["category_sub"] = ""
eco_stat_faq["data_type"] = None
eco_stat_faq["collected_date"] = "2025.02.28"
eco_stat_faq["source_url"] = "https://ecos.bok.or.kr/#/EconomicIndicatorExplanation"
eco_stat_faq["content"] = ""
eco_stat_faq["data_title"] = "주요통계해설_FAQ"
eco_stat_faq["add_info"] = None
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(options=chrome_options)


driver.get("https://ecos.bok.or.kr/#/EconomicIndicatorExplanation")
time.sleep(3)

img_count = 1
add_info_list = []
len_html = 0
text_list = []
key_order = ["제목", "질문", "답변", "metas"]

# faq_text = driver.find_element(By.XPATH, '//*[@id="root"]/div[5]/div/div/div[2]/div/div[2]').text
# faq_num= driver.find_element(By.XPATH, '//*[@id="root"]/div[5]/div/div/div[2]/div/div[2]/div/div[1]/span').text
# num = int(faq_num)

for i in range(12):
    # 왼쪽 nav bar 클릭하기
    left_nav = driver.find_element(By.XPATH, f'//*[@id="root"]/div[5]/div/div/div[1]/ul/li[{i + 1}]/button')
    driver.execute_script("arguments[0].click();", left_nav)
    title = left_nav.text
    time.sleep(3)
    # FAQ 탭 클릭
    explanation = driver.find_element(By.XPATH, '//*[@id="root"]/div[5]/div/div/div[2]/div/div[1]/ul/li[5]/button')
    driver.execute_script("arguments[0].click();", explanation)
    time.sleep(1)
    # <ul class='ex-faq-list'> 내부에서 모든 <li class='ex-faq-item'> 요소 찾기
    ex_faq_items = driver.find_elements(By.XPATH, "//ul[@class='ex-faq-list']//li[@class='ex-faq-item']")

    # 각 토글 active 상태로 변경
    for ex_faq_item in ex_faq_items:
        # "active" 클래스가 없으면 클릭하여 활성화
        if 'active' not in ex_faq_item.get_attribute('class'):
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'})", ex_faq_item)
            ex_faq_item.click()
            time.sleep(1)
    
    # 테이블 스샷찍어 저장
    for body in driver.find_elements(By.CLASS_NAME, "answer-inner"):
        for k, tb_img in enumerate(body.find_elements(By.XPATH, ".//table | .//img"), start=1):
            if tb_img.tag_name == "table":
                print(title, "테이블 >>", f"img_{img_count:03}_{k:03}.png")
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'})", tb_img)
                tb_img.screenshot(f"img_{img_count:03}_{k:03}.png")
            else:
                pass
        img_count += 1

    question = [element.text for element in driver.find_elements(By.CLASS_NAME, "faq-tit")]
    # 각 항목들 html 추출
    htmls = [BeautifulSoup(element.get_attribute("outerHTML"), "html.parser") for element in driver.find_elements(By.CLASS_NAME, "answer-inner")]
    
    # htmls -> 1 GDP의 측정과 3면 등가의 원칙, 2 명목GDP와 실질GDP .....
    # sub_box -> 4-1 국민총소득(GNI), 4-2 국민순소득(NNI) ....
    for l, html in enumerate(htmls, start=1):
        add_info_dict = {}
        add_info_dict["제목"] = title
        add_info_dict["질문"] = question[l - 1]
        add_info_dict["metas"] = []
        for br in html.find_all("br"):
            br.replace_with("\n")
        tb_img_count = 1
        add_info_dict["metas"] = []
        if (html.find("table") == None) and (html.find("img") == None): # 테이블과 이미지 둘 다 없는 경우
            text = "제목: " + title + "\n\n질문: " + question[l - 1] + "\n\n답변: " + html.get_text().replace("\xa0", " ")
            text_list.append(text)
        elif (html.find("table") != None) and (html.find("img") == None): # 테이블만 있고 이미지는 없는 경우
            for table in html.find_all("table"):
                print(title, l, question[l - 1],"table 여기", f"img_{l + len_html:03}_{tb_img_count:03}")                
                metas = {}
                metas["type"] = "table"
                metas["tag"] = f"img_{len_html:03}_{tb_img_count:03}"
                metas["file_name"] = f"img_{l + len_html:03}_{tb_img_count:03}.png"
                image = Image.open(f"img_{l + len_html:03}_{tb_img_count:03}.png")
                metas["img_size"] = {"channel": len(image.mode), "height": image.height, "width": image.width}
                metas["text"] = table.get_text().replace("\xa0", " ")
                add_info_dict["metas"].append(metas)
                table.replace_with(f"<img_{l + len_html:03}_{tb_img_count:03}>")
                tb_img_count += 1
            text = "제목: " + title + "\n\n질문: " + question[l - 1] + "\n\n답변: " + html.get_text().replace("\xa0", " ")
            text_list.append(text)
        elif (html.find("table") == None) and (html.find("img") != None): # 테이블은 없고 이미지만 있는 경우
            # 이미지 저장하기
            for img in html.find_all("img"):
                print(title, l, question[l - 1],"이미지만 여기", f"img_{l + len_html:03}_{tb_img_count:03}")
                try:
                    response = urlopen(img["src"])
                    content = response.read()
                except ValueError:
                    src = "https://ecos.bok.or.kr" + img["src"]
                    response = urlopen(src)
                    content = response.read()
                img_extension = "png"
                file_name = f"img_{l + len_html:03}_{tb_img_count:03}.{img_extension}"
                img_path = os.path.join(r"C:\VScode_test\ecos.bok.or.kr\FAQ", file_name)
                with open(img_path, "wb") as file:
                    file.write(content)
                    # print("이미지 저장 :", f"img_{l + len_html:03}_{tb_img_count:03}")
                metas = {}
                metas["type"] = "image"
                metas["tag"] = f"img_{l + len_html:03}_{tb_img_count:03}"
                metas["file_name"] = f"img_{l + len_html:03}_{tb_img_count:03}.png"
                image = Image.open(f"img_{l + len_html:03}_{tb_img_count:03}.png")
                metas["img_size"] = {"channel": len(image.mode), "height": image.height, "width": image.width}
                add_info_dict["metas"].append(metas)
                
                img.replace_with(f"<img_{l + len_html:03}_{tb_img_count:03}>")
                tb_img_count += 1
            text = "제목: " + title  + "\n\n질문: " + question[l - 1] + "\n\n답변: " + html.get_text().replace("\xa0", " ")
            text_list.append(text)
        else: # 테이블과 이미지 둘 다 있는 경우
            table_text = []
            for table_and_img in html.find_all(lambda tag: tag.name == "table" or tag.name == "img"):
                if table_and_img.name == "table":
                    print(title, l, question[l - 1],"테이블, 이미지 둘 중 테이블", f"img_{l + len_html:03}_{tb_img_count:03}")
                    metas = {}
                    metas["type"] = "table, image"
                    metas["tag"] = f"img_{l + len_html:03}_{tb_img_count:03}"
                    metas["file_name"] = f"img_{l + len_html:03}_{tb_img_count:03}.png"
                    image = Image.open(f"img_{l + len_html:03}_{tb_img_count:03}.png")
                    metas["img_size"] = {"channel": len(image.mode), "height": image.height, "width": image.width}
                    metas["text"] = table_and_img.append(table_and_img.get_text().replace("\xa0", " "))
                    add_info_dict["metas"].append(metas)
                    
                    table_and_img.replace_with(f"<img_{l + len_html:03}_{tb_img_count:03}>")

                else:
                    print(title, l, question[l - 1],"테이블, 이미지 둘 중 이미지", f"img_{l + len_html:03}_{tb_img_count:03}")
                    try:
                        response = urlopen(table_and_img["src"])
                        content = response.read()
                    except ValueError:
                        src = "https://ecos.bok.or.kr" + table_and_img["src"]
                        response = urlopen(src)
                        content = response.read()
                    img_extension = "png"
                    file_name = f"img_{l + len_html:03}_{tb_img_count:03}.{img_extension}"
                    img_path = os.path.join(r"C:\VScode_test\ecos.bok.or.kr\FAQ", file_name)
                    with open(img_path, "wb") as file:
                        file.write(content)
                    metas = {}
                    metas["type"] = "image"
                    metas["tag"] = f"img_{l + len_html:03}_{tb_img_count:03}"
                    metas["file_name"] = f"img_{l + len_html:03}_{tb_img_count:03}.png"
                    image = Image.open(f"img_{l + len_html:03}_{tb_img_count:03}.png")
                    metas["img_size"] = {"channel": len(image.mode), "height": image.height, "width": image.width}
                    add_info_dict["metas"].append(metas)
                    
                    table_and_img.replace_with(f"<img_{l + len_html:03}_{tb_img_count:03}>")
                tb_img_count += 1
            text = "제목: " + title + "\n\n질문: " + question[l - 1] + "\n\n답변: " + html.get_text().replace("\xa0", " ")
            text_list.append(text)
        add_info_dict["답변"] = html.get_text().replace("\xa0", " ")
        add_info_dict = dict(sorted(add_info_dict.items(), key=lambda k: key_order.index(k[0])))
        # eco_stat_faq.loc[(l - 1) + , "content"] = text
        add_info_list.append(add_info_dict)
    # if len_html == 0:
    #     len_html -= 1
    len_html += len(htmls)
        

국민소득통계 테이블 >> img_005_001.png
국민소득통계 테이블 >> img_009_002.png
국민소득통계 5 GNP는 더 이상 작성하지 않는 지표인가요? table 여기 img_005_001
국민소득통계 7 경제성장률은 어떻게 산출하나요? 이미지만 여기 img_007_001
국민소득통계 7 경제성장률은 어떻게 산출하나요? 이미지만 여기 img_007_002
국민소득통계 8 분기 경제성장률에는 전기비, 전년동기비, 연율 등 여러 종류가 있는데, 차이는 무엇인지요? 이미지만 여기 img_008_001
국민소득통계 8 분기 경제성장률에는 전기비, 전년동기비, 연율 등 여러 종류가 있는데, 차이는 무엇인지요? 이미지만 여기 img_008_002
국민소득통계 8 분기 경제성장률에는 전기비, 전년동기비, 연율 등 여러 종류가 있는데, 차이는 무엇인지요? 이미지만 여기 img_008_003
국민소득통계 8 분기 경제성장률에는 전기비, 전년동기비, 연율 등 여러 종류가 있는데, 차이는 무엇인지요? 이미지만 여기 img_008_004
국민소득통계 8 분기 경제성장률에는 전기비, 전년동기비, 연율 등 여러 종류가 있는데, 차이는 무엇인지요? 이미지만 여기 img_008_005
국민소득통계 8 분기 경제성장률에는 전기비, 전년동기비, 연율 등 여러 종류가 있는데, 차이는 무엇인지요? 이미지만 여기 img_008_006
국민소득통계 8 분기 경제성장률에는 전기비, 전년동기비, 연율 등 여러 종류가 있는데, 차이는 무엇인지요? 이미지만 여기 img_008_007
국민소득통계 9 계절변동조정이란 무엇인가요? 테이블, 이미지 둘 중 이미지 img_009_001
국민소득통계 9 계절변동조정이란 무엇인가요? 테이블, 이미지 둘 중 테이블 img_009_002
국민소득통계 13 기여도와 기여율은 어떻게 계산하나요? 이미지만 여기 img_013_001
국민소득통계 13 기여도와 기여율은 어떻게 계산하나요? 이미지만 여기 img_013_002
산업연관표 테이블 >> img_017_

In [4]:
len_html

94

In [5]:
for i in range(len(add_info_list)):
    if add_info_list[i]["metas"] == []:
        del add_info_list[i]["metas"]

eco_stat_faq.loc[:93, "content"] = text_list
eco_stat_faq.loc[:93, "add_info"] = add_info_list
eco_stat_faq = eco_stat_faq[eco_stat_faq.content != ""]

for i in range(len(eco_stat_faq)):
    if "metas" in eco_stat_faq.add_info.iloc[i].keys():
        eco_stat_faq.at[i, "data_type"] = ["text", "image"]
    else:
        eco_stat_faq.at[i, "data_type"] = ["text"]

In [6]:
# jsonl 추출
eco_stat_faq.to_json("eco_stat_faq.jsonl", orient="records", force_ascii=False, lines=True)

In [ ]:
driver.execute_script(f"document.body.style.zoom='{zoom}%")


def is_fully_visible(driver, element):
    '''
    element(table)가 laptop 화면에서 완전히 보이는지 잘려서 보이는지를 반환해주는 함수
    완전히 보일 시 -> True 반환
    잘려서 보일 시 -> False 반환
    '''
    return driver.execute_script(
        "var rect = arguments[0].getBoundingClientRect();"
        "return (rect.top >= 0 && rect.left >= 0 && "
        "rect.bottom <= (window.innerHeight || document.documentElement.clientHeight) && "
        "rect.right <= (window.innerWidth || document.documentElement.clientWidth));",
        element
    )

# 테이블 스샷찍어 저장
    for body in driver.find_elements(By.CLASS_NAME, "answer-inner"):
        for k, tb_img in enumerate(body.find_elements(By.XPATH, ".//table | .//img"), start=1):
            if tb_img.tag_name == "table":
                print(title, "테이블 >>", f"img_{img_count:03}_{k:03}.png")
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'})", tb_img)
                tb_img.screenshot(f"img_{img_count:03}_{k:03}.png")
            else:
                pass
        img_count += 1

# element를 브라우저 가운데로 보이도록 스크롤해주는 코드
    driver.execute_script("""
        arguments[0].scrollIntoView({
            behavior: 'auto',
            block: 'center',
            inline: 'center'
        });
    """, element)
driver.execute_script(f"document.body.style.zoom='{zoom}%")

위 코드를 사용하여, table를 screenshot할 때 완전히 보이지 않으면 zoom을 축소하여 완전히 보이게 한 뒤 캡쳐할거야.
기본 zoom는 100%이며, zoom를 하여 캡쳐한 경우 다시 100%로 돌아가야합니다.
while문을 써서 table이 완전히 보일때까지 zoom를 해주세요. (min_zoom=50%)